In [147]:
# Install necessary libraries
!pip install ipywidgets matplotlib pandas firebase seaborn numpy pytz

# Import required libraries and modules
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
from firebase import firebase
import seaborn as sns
import numpy as np
import matplotlib.dates as mdates
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os
import nltk
from nltk.chat.util import Chat, reflections
from collections import Counter
import pytz

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

###############################
##### FIREBASE CONNECTION #####
###############################
# Firebase URL
firebase_url = 'https://hw2-tiger-default-rtdb.europe-west1.firebasedatabase.app/'

# Initialize the connection to Firebase
FBconn = firebase.FirebaseApplication(firebase_url, None)

# Function to fetch data from Firebase
def fetch_data_from_firebase():
    """
    This function fetches data from the specified Firebase database.

    It connects to the Firebase URL and retrieves data stored under the '/HW2-Tiger/' path.
    The retrieved data is stored in a list and returned.
    If no data is found, an empty list is returned.
    """
    result = FBconn.get('/HW2-Tiger/', None)
    if result:
        data = [value for value in result.values()]
        return data, result
    else:
        return []

#################################
##### EMAIL SENDING SERVICE #####
#################################
# Email configuration
EMAIL_ADDRESS = 'uriziv123654@gmail.com'  # Your email address
EMAIL_PASSWORD = 'yqce zdce tqrn wslc'  # Your app-specific password
SMTP_SERVER = 'smtp.gmail.com'  # SMTP server for Gmail
SMTP_PORT = 587  # Port number for Gmail

# Function to send email with an attachment
def send_email_with_attachment(subject, body, attachment_path, recipient_email):
    """
    This function sends an email with an attachment.

    It uses the smtplib library to send an email from a Gmail account.
    The email includes a subject, body, and an attachment.
    """
    msg = MIMEMultipart()
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = recipient_email
    msg['Subject'] = subject

    # Attach the email body
    msg.attach(MIMEText(body, 'html'))

    # Attach the file
    attachment = MIMEBase('application', 'octet-stream')
    with open(attachment_path, 'rb') as file:
        attachment.set_payload(file.read())
    encoders.encode_base64(attachment)
    attachment.add_header('Content-Disposition', f'attachment; filename={os.path.basename(attachment_path)}')
    msg.attach(attachment)

    # Connect to the SMTP server and send the email
    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.starttls()
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        server.send_message(msg)

# Function to create the send email button and email input field
def create_send_email_button(image_path, subject):
    """
    This function creates a button and an input field for sending an email with an attachment.

    When the button is clicked, the email is sent to the address entered in the input field.
    """
    send_email_button = widgets.Button(
        description='Send Email',
        layout=widgets.Layout(width='200px', height='50px'),
    )

    email_input = widgets.Text(
        description='Email:',
        layout=widgets.Layout(width='400px')
    )

    def on_send_email_button_clicked(b):
        """
        This function is called when the send email button is clicked.

        It retrieves the email address from the input field and sends an email with the specified attachment.
        """
        recipient_email = email_input.value
        if recipient_email:
            body = f"<p>Please find the attached graph image for {subject}.</p>"
            send_email_with_attachment(subject, body, image_path, recipient_email)
            with plot_output:
                message = f"Email sent to {recipient_email} with the attached graph image for {subject}."
                show_popup(message, title="Message")
        else:
            with plot_output:
                display(widgets.HTML("<p>Please enter a valid email address.</p>"))

    send_email_button.on_click(on_send_email_button_clicked)
    return widgets.HBox([email_input, send_email_button], layout=widgets.Layout(align_items='center'))

###########################
##### STYLE FUNCTIONS #####
###########################

# Set a greenish color palette using seaborn
green_palette = sns.color_palette("Greens", as_cmap=True)

# Function to show a popup message
def show_popup(message, title="Message"):
    """
    This function displays a popup message using ipywidgets.

    The popup contains a title, message, and a close button.
    """
    popup = widgets.Output()
    with popup:
        clear_output()
        display(widgets.HTML(f"""
        <div style='background-color: white; text-color: black; padding: 20px; border-radius: 50px; border: 1px solid black; max-width: 400px; text-align: center;'>
            <h2 style='color: black;'>{title}</h2>
            <p style='color: black'>{message}</p>
            <button onclick="this.parentElement.parentElement.style.display='none';">Close</button>
        </div>
        """))
    display(popup)

##################################################
##### DASHBOARD AND PAGES CREATING FUNCTIONS #####
##################################################

# Function to create the main dashboard menu
def create_main_dashboard():
    """
    This function creates the main dashboard with buttons for different actions.

    The buttons are arranged in a grid layout.
    """
    # Create buttons for the dashboard
    dashboard_buttons = [
        widgets.Button(description='Actions by User', layout=widgets.Layout(width='200px', height='75px')),
        widgets.Button(description='Actions by Document', layout=widgets.Layout(width='200px', height='75px')),
        widgets.Button(description='Number of Actions Over Time', layout=widgets.Layout(width='200px', height='75px')),
        widgets.Button(description='Actions by Tab', layout=widgets.Layout(width='200px', height='75px')),
        widgets.Button(description='Personal Assistant', layout=widgets.Layout(width='200px', height='75px')),
        widgets.Button(description='Progress Patterns', layout=widgets.Layout(width='200px', height='75px')),
        widgets.Button(description='Collaboration Signs', layout=widgets.Layout(width='200px', height='75px')),
        widgets.Button(description='Working Hours', layout=widgets.Layout(width='200px', height='75px')),
        widgets.Button(description='User Contributions', layout=widgets.Layout(width='200px', height='75px')),
        widgets.Button(description='Student Activities', layout=widgets.Layout(width='200px', height='75px'))
    ]

    # Arrange buttons in a grid
    dashboard_grid = widgets.GridBox(
        children=dashboard_buttons,
        layout=widgets.Layout(
            width='100%',
            grid_template_columns='repeat(5, 250px)',
            justify_content='center',
            align_items='center',
            grid_gap='10px',
            padding='20px'
        )
    )

    # Title HTML widget
    title = widgets.HTML(value=f"""
        <div style='background: linear-gradient(to right, #ffffff, #6EC373, #ffffff); padding: 20px; border-radius: 10px; color: black; text-align: center;'>
            <h1 style='color: black'>Welcome to OnShape, Amir Cohen</h1>
            <h2 style='color: black'>{(datetime.now(pytz.timezone('Asia/Jerusalem'))).strftime('%Y-%m-%d %H:%M:%S')}</h2>
            <h3 style='color: black'>On the dashboard, you can explore different aspects of your project management data. Check out actions by user, see which documents are accessed the most, track the number of actions over time, and dive into actions by tab. It's all about giving you a clear view of what's happening in your project!</h3>
        </div>
    """)

    # Combine title and grid into a VBox with a white background
    dashboard_content = widgets.VBox([title, dashboard_grid], layout=widgets.Layout(
        padding='20px',
        width='100%',
        background_color='white'
    ))

    # Wrap the content in a Box to set the background color and ensure full width
    dashboard_box = widgets.Box([dashboard_content], layout=widgets.Layout(
        width='100%',
        height='auto',
        background_color='white',
        padding='20px'
    ))

    # Function to navigate to the dashboard (placeholder, can be customized)
    def go_to_dashboard(button):
        display_main_dashboard()

    # Assign the navigation function to all buttons
    for button in dashboard_buttons:
        button.on_click(go_to_dashboard)

    return dashboard_box

# Function to display the main dashboard
def display_main_dashboard():
    """
    This function displays the main dashboard with buttons for different actions.

    It clears the previous output and displays the dashboard content.
    """
    clear_output()
    plot_output.clear_output()  # Clear plot output when navigating to main dashboard
    dashboard_content = create_main_dashboard()

    # Access the GridBox which is the second child of the VBox
    dashboard_grid = dashboard_content.children[0].children[1]

    # Assign on_click events to the buttons
    dashboard_grid.children[0].on_click(lambda b: display_actions_by_user())
    dashboard_grid.children[1].on_click(lambda b: display_actions_by_document())
    dashboard_grid.children[2].on_click(lambda b: display_number_of_actions_over_time())
    dashboard_grid.children[3].on_click(lambda b: display_actions_by_tab())
    dashboard_grid.children[4].on_click(lambda b: display_virtual_assistant())
    dashboard_grid.children[5].on_click(lambda b: display_progress_patterns())
    dashboard_grid.children[6].on_click(lambda b: display_collaboration_signs())
    dashboard_grid.children[7].on_click(lambda b: display_working_hours())
    dashboard_grid.children[8].on_click(lambda b: display_user_contributions())
    dashboard_grid.children[9].on_click(lambda b: display_student_activities())

    display(dashboard_content)

# Placeholder for plots
plot_output = widgets.Output()

# Function to create individual pages
def create_page(title_text, description, additional_widgets=None):
    """
    This function creates an individual page with a title and optional additional widgets.

    The page includes a title, a back button, and any additional widgets passed as arguments.
    """
    page_title = widgets.HTML(value=f"""
        <div style='background: linear-gradient(to right, #ffffff, #6EC373, #ffffff); padding: 10px; width: full; border-radius: 10px; text-align: center;'>
            <h1 style='color: black'>{title_text}</h1>
            <h3 style='color: black'>{description}</h3>
        </div>
    """)

    seperator = widgets.HTML(value=f"""
        <div style='height: 20px'></div>
    """)

    back_button = widgets.Button(description='Back to Dashboard', layout=widgets.Layout(width='200px', height='50px'))

    # Function to navigate back to the main dashboard
    def back_to_dashboard(b):
        plot_output.clear_output()  # Clear plot output when navigating back to dashboard
        display_main_dashboard()

    back_button.on_click(back_to_dashboard)

    # Combine title, back button, and any additional widgets into a single layout
    children = [back_button, seperator, page_title, seperator]
    if additional_widgets:
        children.extend(additional_widgets)

    page_content = widgets.VBox(children, layout=widgets.Layout(
        width='100%',
        height='auto',
        background_color='white',
        align_items='center',
        padding='20px'
    ))

    # Wrap the content in a Box to set the background color and ensure full width
    page_box = widgets.Box([page_content], layout=widgets.Layout(
        width='100%',
        padding='20px',
        background_color='white'
    ))

    return page_box

#############################
### UTILITIES FUNCTIONS #####
#############################

# Utility function to truncate labels for better readability
def truncate_labels(labels, max_length=25):
    """
    This function truncates labels to a maximum length.

    If a label is longer than max_length, it is truncated and "..." is added at the end.
    """
    truncated = [label if len(label) <= max_length else label[:max_length-3] + "..." for label in labels]
    return truncated

##########################
### PLOTTING METHODS #####
##########################

actions_by_user_description = "This pie chart illustrates the distribution of actions performed by various users. Each segment of the pie represents a user, with the size of the segment corresponding to the proportion of actions they have performed out of the total. The percentage labels on the chart indicate the exact share of actions attributed to each user, providing a clear visual representation of user activity distribution. This dynamically generated chart can help identify which users are most active and contribute the most actions."
actions_by_document_description = "This bar chart illustrates the distribution of actions across different documents. Each bar represents a specific document, with the height of the bar corresponding to the number of actions performed on that document. The x-axis lists the document names, while the y-axis indicates the number of actions. This dynamically generated chart provides a clear visual representation of which documents are the most frequently interacted with, helping to identify the focus areas and activity levels for each document."
actions_over_time_description = "This line graph shows the number of actions performed over the project's life cycle. The x-axis represents the timeline, while the y-axis indicates the number of actions. Each point on the line represents the number of actions at a given time. This dynamically generated graph helps to visualize trends and patterns in activity over time, highlighting periods of high and low activity."
actions_by_tab_description = "This bar chart displays the number of actions performed across different tabs within a document. Each bar represents a specific tab, with the height of the bar indicating the number of actions associated with that tab. The x-axis lists the tab names, while the y-axis shows the number of actions. This dynamically generated chart provides insight into which tabs are most frequently interacted with, helping to identify areas of focus and activity within the document."
virtual_assistant_description = "The Project Management Assistant chatbot is designed to help users manage and track their project activities efficiently. It can provide insights into user actions, document accesses, and tab usage within a project by fetching and analyzing data from the database. The chatbot responds to various queries about project management tasks, user statistics, and action descriptions."
progress_patterns_description = "This graph visualizes the progress pattern of new tabs and features added by users over time. The stacked bar chart shows the contributions of each user, with different colors representing tabs and features, allowing for easy identification of individual user activity and overall progress."
collaboration_signs_description = "This graph shows the signs of collaboration among users by highlighting overlapping contributions to the same tabs during the same hours. Each point represents a collaborative effort by multiple users on a specific tab at a particular hour, with the legend indicating the tabs and number of users involved"
working_hours_description = "This bar chart shows the working hours of each student, categorized into regular days, night shifts, and weekends. Each bar represents the number of actions performed by a student in each category, highlighting their activity patterns."
user_contributions_description = "This stacked bar chart shows the contribution of each user to the progress of the task, categorized by commit types (descriptions). Each bar represents a type of commit, with segments for each user, highlighting their individual contributions."
student_activities_description = "This stacked bar chart shows the main activities of each student, categorized into Creative, Viewing, and Administrative. Each bar represents a student, with segments for each activity type, highlighting their individual contributions to different tasks. Below the graph, the following information is displayed: The user with the most contributions in total, The user with the most creative contributions, The user with the lowest contributions in total, The user with the most administrative actions, The user with the most viewing actions."


# Plotting functions with save functionality
def plot_actions_by_user(save_path='actions_by_user.png'):
    """
    This function plots the number of actions by user and saves the plot as an image.

    It fetches data from Firebase, creates a pie chart showing the number of actions by each user,
    and saves the plot as a PNG file.
    """
    data, _ = fetch_data_from_firebase()
    if data:
        df = pd.DataFrame(data)
        with plot_output:
            plot_output.clear_output()
            fig, ax = plt.subplots(figsize=(6, 4))  # Adjust the figsize for better layout
            colors = sns.color_palette("Greens", as_cmap=True)(np.linspace(0.5, 1, df['User'].nunique()))  # Apply the color palette

            user_counts = df['User'].value_counts()
            user_counts.plot(kind='pie', ax=ax, colors=colors, autopct='%1.1f%%', startangle=90)
            ax.set_title('Number of Actions by User', fontsize=16)
            ax.set_ylabel('')  # Remove y-label for a cleaner look
            ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
            plt.show()
            fig.savefig(save_path, format='png')  # Save the plot as an image

def plot_actions_by_document(save_path='actions_by_document.png'):
    """
    This function plots the number of actions by document and saves the plot as an image.

    It fetches data from Firebase, creates a bar chart showing the number of actions for each document,
    and saves the plot as a PNG file.
    """
    data, _ = fetch_data_from_firebase()
    if data:
        df = pd.DataFrame(data)
        with plot_output:
            plot_output.clear_output()
            fig, ax = plt.subplots(figsize=(6, 4))  # Adjust the figsize for better layout
            colors = sns.color_palette("Greens", as_cmap=True)(np.linspace(0.5, 1, df['Document'].nunique()))  # Apply the color palette

            document_counts = df['Document'].value_counts()
            truncated_labels = truncate_labels(document_counts.index)

            document_counts.index = truncated_labels  # Update the labels with truncated versions

            document_counts.plot(kind='bar', ax=ax, color=colors)
            ax.set_title('Number of Actions by Document', fontsize=16)
            ax.set_xlabel('Document', fontsize=14)
            ax.set_ylabel('Number of Actions', fontsize=14)
            plt.xticks(rotation=45, ha='right', fontsize=12)  # Rotate and adjust label font size
            plt.yticks(fontsize=12)
            ax.grid(True, which='both', linestyle='--', linewidth=0.5)
            plt.show()
            fig.savefig(save_path, format='png')  # Save the plot as an image

def plot_number_of_actions_over_time(save_path='number_of_actions_over_time.png'):
    """
    This function plots the number of actions over time and saves the plot as an image.

    It fetches data from Firebase, resamples it to get daily action counts,
    creates a line chart showing the number of actions over time, and saves the plot as a PNG file.
    """
    data, _ = fetch_data_from_firebase()
    if data:
        df = pd.DataFrame(data)
        df['Time'] = pd.to_datetime(df['Time'], errors='coerce')  # Coerce errors to NaT
        df = df.dropna(subset=['Time'])  # Drop rows with NaT in 'Time'
        df = df[df['Time'] > datetime(2000, 1, 1)]  # Remove any erroneous old dates

        df_resampled = df.resample('D', on='Time').size()

        with plot_output:
            plot_output.clear_output()
            fig, ax = plt.subplots(figsize=(8, 4))  # Adjust the figsize for better layout
            colors = sns.color_palette("Greens", as_cmap=True)(np.linspace(0.5, 1, len(df_resampled)))  # Apply the color palette
            df_resampled.plot(kind='line', ax=ax, color=colors[0])  # Use the first color for the line
            ax.set_title('Number of Actions Over Time', fontsize=16)
            ax.set_xlabel('Time', fontsize=14)
            ax.set_ylabel('Number of Actions', fontsize=14)

            # Improve x-axis date formatting
            ax.xaxis.set_major_locator(mdates.MonthLocator())
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
            plt.xticks(rotation=90, ha='right', fontsize=12)

            plt.yticks(fontsize=12)
            ax.grid(True, which='both', linestyle='--', linewidth=0.5)
            plt.show()
            fig.savefig(save_path, format='png')  # Save the plot as an image

def plot_actions_by_tab(document_name, save_path='actions_by_tab.png'):
    """
    This function plots the number of actions by tab for a specific document and saves the plot as an image.

    It fetches data from Firebase, filters it for the specified document,
    creates a bar chart showing the number of actions for each tab, and saves the plot as a PNG file.
    """
    data, _ = fetch_data_from_firebase()
    if data:
        df = pd.DataFrame(data)
        df = df[df['Document'] == document_name].copy()
        df.loc[df['Tab'] == 'N/A', 'Tab'] = 'Other'
        with plot_output:
            plot_output.clear_output()
            fig, ax = plt.subplots(figsize=(8, 4))  # Adjust the figsize for better layout

            tab_counts = df['Tab'].value_counts()
            truncated_labels = truncate_labels(tab_counts.index)

            tab_counts.index = truncated_labels  # Update the labels with truncated versions

            colors = sns.color_palette("Greens", as_cmap=True)(np.linspace(0.5, 1, tab_counts.nunique()))  # Apply the color palette

            tab_counts.plot(kind='bar', ax=ax, color=colors)
            ax.set_title(f'Number of Actions by Tab for {document_name}', fontsize=16)
            ax.set_xlabel('Tab', fontsize=14)
            ax.set_ylabel('Number of Actions', fontsize=14)
            plt.xticks(rotation=45, ha='right', fontsize=12)  # Rotate and adjust label font size
            plt.yticks(fontsize=12)
            ax.grid(True, which='both', linestyle='--', linewidth=0.5)
            plt.show()
            fig.savefig(save_path, format='png')  # Save the plot as an image

# Function to display the dropdown menu for selecting a document
def display_document_selection():
    """
    This function displays a dropdown menu for selecting a document.

    The selected document is used to create a plot of actions by tab.
    """
    data, _ = fetch_data_from_firebase()
    if data:
        df = pd.DataFrame(data)
        # Remove NaN values before creating the list of options
        documents = ['Choose...'] + list(df['Document'].dropna().unique())
        document_selector = widgets.Dropdown(
            options=documents,
            description='Select Doc:',
            layout=widgets.Layout(width='500px')
        )

        def on_document_selected(change):
            """
            This function is called when a document is selected from the dropdown menu.

            It creates a plot of actions by tab for the selected document.
            """
            if change['type'] == 'change' and change['name'] == 'value':
                if change['new'] == 'Choose...':
                    plot_output.clear_output()
                else:
                    display_plot_with_send_email(lambda path: plot_actions_by_tab(change['new'], path), 'actions_by_tab.png', f'Actions by Tab for {change["new"]}', "")

        document_selector.observe(on_document_selected)

        additional_widgets = [document_selector]
        clear_output()
        page = create_page('Actions by Tab', actions_by_tab_description, additional_widgets)
        display(page)
        display(plot_output)

def plot_progress_pattern(save_path='progress_pattern.png'):
    """
    This function plots the progress pattern of new tabs added by users over time and saves the plot as an image.
    """
    data, _ = fetch_data_from_firebase()
    df = pd.DataFrame(data)
    df['Time'] = pd.to_datetime(df['Time'], errors='coerce')  # Coerce errors to NaT
    df = df.dropna(subset=['Time'])  # Drop rows with NaT in 'Time'
    df = df[df['Time'] > datetime(2000, 1, 1)]  # Remove any erroneous old dates

    # Filter for new tabs and features
    new_tabs = df[df['Description'].str.contains('Tab') & df['Description'].str.contains('created', case=False)]
    add_features = df[df['Description'].str.contains('feature', case=False) | df['Description'].str.contains('Add or modify a sketch', case=False)]

    # Aggregate data
    tabs_over_time = new_tabs.groupby([new_tabs['Time'].dt.date, 'User']).size().unstack(fill_value=0)
    features_over_time = add_features.groupby([add_features['Time'].dt.date, 'User']).size().unstack(fill_value=0)

    with plot_output:
        plot_output.clear_output()
        fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figsize for better layout

        # Plot stacked bar charts with individual labels
        for user in tabs_over_time.columns:
            ax.bar(tabs_over_time.index, tabs_over_time[user], label=f"{user} - Tabs", alpha=0.7, width=0.4)

        for user in features_over_time.columns:
            ax.bar(features_over_time.index, features_over_time[user], label=f"{user} - Features", alpha=0.7, width=0.4, hatch='//')

        ax.set_title('Progress Pattern: New Tabs and Features by User Over Time', fontsize=16)
        ax.set_xlabel('Time', fontsize=14)
        ax.set_ylabel('Count', fontsize=14)
        plt.xticks(rotation=45, ha='right', fontsize=12)

        plt.yticks(fontsize=12)

        # Set the legend with new graph labels
        ax.legend(title='User Activity', bbox_to_anchor=(1.05, 1), loc='upper left')

        ax.grid(True, which='both', linestyle='--', linewidth=0.5)
        plt.tight_layout()
        plt.show()
        fig.savefig(save_path, format='png')  # Save the plot as an image

def plot_collaboration_signs(save_path='collaboration_signs.png'):
    """
    This function plots the signs of collaboration by showing overlapping contributions by users to the same tabs during the same hours, and saves the plot as an image.
    """
    data, _ = fetch_data_from_firebase()
    df = pd.DataFrame(data)
    df['Time'] = pd.to_datetime(df['Time'], errors='coerce')  # Coerce errors to NaT
    df = df.dropna(subset=['Time'])  # Drop rows with NaT in 'Time'
    df = df[df['Time'] > datetime(2000, 1, 1)]  # Remove any erroneous old dates

    # Extract hour and date for easier analysis
    df['Hour'] = df['Time'].dt.hour
    df['Date'] = df['Time'].dt.date

    # Find overlapping contributions by grouping by Tab, Date, and Hour, then checking the number of unique users
    collaboration = df.groupby(['Tab', 'Date', 'Hour']).User.nunique().reset_index()
    collaboration = collaboration[collaboration['User'] > 1]

    with plot_output:
        plot_output.clear_output()
        fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figsize for better layout

        for index, row in collaboration.iterrows():
            ax.plot(row['Date'], row['Hour'], 'o', label=f"Tab: {row['Tab']}, Users: {row['User']}")

        ax.set_title('Collaboration Signs: Overlapping Contributions by Users', fontsize=16)
        ax.set_xlabel('Date', fontsize=14)
        ax.set_ylabel('Hour', fontsize=14)
        plt.xticks(rotation=45, ha='right', fontsize=12)

        plt.yticks(fontsize=12)

        # Set the legend with new graph labels
        handles, labels = ax.get_legend_handles_labels()
        unique_labels = list(set(labels))  # Remove duplicate labels
        ax.legend(handles[:len(unique_labels)], unique_labels, title='Collaborations', bbox_to_anchor=(1.05, 1), loc='upper left')

        ax.grid(True, which='both', linestyle='--', linewidth=0.5)
        plt.tight_layout()
        plt.show()
        fig.savefig(save_path, format='png')  # Save the plot as an image

def plot_working_hours(save_path='working_hours.png'):
    """
    This function plots the working hours of each student and highlights occurrences of working during night, weekends, and holidays. The plot is saved as an image.
    """
    data, _ = fetch_data_from_firebase()
    df = pd.DataFrame(data)
    df['Time'] = pd.to_datetime(df['Time'], errors='coerce')  # Coerce errors to NaT
    df = df.dropna(subset=['Time'])  # Drop rows with NaT in 'Time'
    df = df[df['Time'] > datetime(2000, 1, 1)]  # Remove any erroneous old dates

    # Extract hour and day of the week
    df['Hour'] = df['Time'].dt.hour
    df['DayOfWeek'] = df['Time'].dt.dayofweek  # Monday=0, Sunday=6

    # Identify night hours (e.g., 22:00 to 06:00)
    df['Shift'] = df['Hour'].apply(lambda x: 'Night Shift' if x >= 22 or x < 6 else 'Day Shift')

    # Identify weekends (Friday and Saturday)
    df['Weekend'] = df['DayOfWeek'].apply(lambda x: 'Weekend' if x >= 4 else 'Weekday')

    # Combine shift and weekend information
    df['ShiftType'] = df.apply(lambda row: f"{row['Shift']}, {row['Weekend']}", axis=1)

    # Classify into categories
    def classify_shift(row):
        if row['Shift'] == 'Night Shift':
            return 'Night Shifts'
        elif row['Weekend'] == 'Weekend':
            return 'Weekends'
        else:
            return 'Regular Days'

    df['ShiftCategory'] = df.apply(classify_shift, axis=1)

    # Aggregate data by user and shift category
    working_hours = df.groupby(['User', 'ShiftCategory']).size().unstack(fill_value=0)

    with plot_output:
        plot_output.clear_output()
        fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figsize for better layout

        # Plot data
        working_hours.plot(kind='bar', ax=ax, color=sns.color_palette("Paired"))

        ax.set_title('Working Hours of Each Student', fontsize=16)
        ax.set_xlabel('Shift Category', fontsize=14)
        ax.set_ylabel('Number of Actions', fontsize=14)
        plt.xticks(rotation=0, ha='center', fontsize=12)

        plt.yticks(fontsize=12)

        # Set the legend with user labels
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles, labels, title='User', bbox_to_anchor=(1.05, 1), loc='upper left')

        ax.grid(True, which='both', linestyle='--', linewidth=0.5)
        plt.tight_layout()
        plt.show()
        fig.savefig(save_path, format='png')  # Save the plot as an image

def plot_user_contributions(save_path='user_contributions.png'):
    """
    This function plots the contribution of each user to the progress of the task, categorized by commit types (descriptions), using a stacked bar chart. The plot is saved as an image.
    """
    data, _ = fetch_data_from_firebase()
    df = pd.DataFrame(data)
    df['Time'] = pd.to_datetime(df['Time'], errors='coerce')  # Coerce errors to NaT
    df = df.dropna(subset=['Time'])  # Drop rows with NaT in 'Time'
    df = df[df['Time'] > datetime(2000, 1, 1)]  # Remove any erroneous old dates

    # Count the occurrences of each description
    top_descriptions = df['Description'].value_counts().head(10).index

    # Filter the dataframe to include only the top 10 descriptions
    df_top = df[df['Description'].isin(top_descriptions)]

    # Aggregate data by description and user
    contribution = df_top.groupby(['Description', 'User']).size().unstack(fill_value=0)

    with plot_output:
        plot_output.clear_output()
        fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figsize for better layout

        # Plot data
        contribution.plot(kind='bar', stacked=True, ax=ax, color=sns.color_palette("Paired"))

        ax.set_title('User Contributions to Task Progress (Top 10 Descriptions)', fontsize=16)
        ax.set_xlabel('Commit Types (Descriptions)', fontsize=14)
        ax.set_ylabel('Number of Actions', fontsize=14)
        plt.xticks(rotation=90, ha='right', fontsize=12)
        plt.yticks(fontsize=12)

        # Set the legend with user labels
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles, labels, title='User', bbox_to_anchor=(1.05, 1), loc='upper left')

        ax.grid(True, which='both', linestyle='--', linewidth=0.5)
        plt.tight_layout()
        plt.show()
        fig.savefig(save_path, format='png')  # Save the plot as an image

def plot_student_activities(save_path='student_activities.png'):
    """
    This function plots the main activities of each student categorized into Creative, Viewing, and Administrative using a stacked bar chart.
    The plot is saved as an image. Below the graph, it also displays the user who made the most contributions in total, the user who made
    the most creative contributions, the user with the lowest contributions in total, the user with the most administrative actions, and
    the user with the most viewing actions.
    """
    data, _ = fetch_data_from_firebase()
    df = pd.DataFrame(data)
    df['Time'] = pd.to_datetime(df['Time'], errors='coerce')  # Coerce errors to NaT
    df = df.dropna(subset=['Time'])  # Drop rows with NaT in 'Time'
    df = df[df['Time'] > datetime(2000, 1, 1)]  # Remove any erroneous old dates

    # Classify activities into categories
    def classify_activity(description):
        if any(keyword in description.lower() for keyword in ['create', 'delete', 'modify']):
            return 'Creative'
        elif any(keyword in description.lower() for keyword in ['open', 'close', 'view']):
            return 'Viewing'
        elif any(keyword in description.lower() for keyword in ['import', 'export']):
            return 'Administrative'
        else:
            return 'Other'

    df['Activity_Type'] = df['Description'].apply(classify_activity)

    # Aggregate data by user and activity type
    activity_summary = df.groupby(['User', 'Activity_Type']).size().unstack(fill_value=0)

    # Find the user with the most contributions in total
    total_contributions = activity_summary.sum(axis=1)
    user_most_contributions = total_contributions.idxmax()
    total_contributions_count = total_contributions.max()

    # Find the user with the most creative contributions
    if 'Creative' in activity_summary.columns:
        creative_contributions = activity_summary['Creative']
        user_most_creative_contributions = creative_contributions.idxmax()
        creative_contributions_count = creative_contributions.max()
    else:
        user_most_creative_contributions = 'N/A'
        creative_contributions_count = 0

    # Find the user with the lowest contributions in total
    user_least_contributions = total_contributions.idxmin()
    least_contributions_count = total_contributions.min()

    # Find the user with the most administrative actions
    if 'Administrative' in activity_summary.columns:
        administrative_contributions = activity_summary['Administrative']
        user_most_administrative_contributions = administrative_contributions.idxmax()
        administrative_contributions_count = administrative_contributions.max()
    else:
        user_most_administrative_contributions = 'N/A'
        administrative_contributions_count = 0

    # Find the user with the most viewing actions
    if 'Viewing' in activity_summary.columns:
        viewing_contributions = activity_summary['Viewing']
        user_most_viewing_contributions = viewing_contributions.idxmax()
        viewing_contributions_count = viewing_contributions.max()
    else:
        user_most_viewing_contributions = 'N/A'
        viewing_contributions_count = 0

    with plot_output:
        plot_output.clear_output()
        fig, ax = plt.subplots(figsize=(10, 4))  # Adjust the figsize for better layout

        # Plot data
        activity_summary.plot(kind='bar', stacked=True, ax=ax, color=sns.color_palette("Paired"))

        ax.set_title('Main Activities of Each Student', fontsize=16)
        ax.set_xlabel('User', fontsize=14)
        ax.set_ylabel('Number of Actions', fontsize=14)
        plt.xticks(rotation=0, ha='center', fontsize=12)
        plt.yticks(fontsize=12)

        # Set the legend with activity type labels
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles, labels, title='Activity Type', bbox_to_anchor=(1.05, 1), loc='upper left')

        ax.grid(True, which='both', linestyle='--', linewidth=0.5)
        plt.tight_layout()

        # Add text below the plot
        plt.figtext(0.1, -0.05, f"User with the most contributions in total: {user_most_contributions} ({total_contributions_count} actions)", fontsize=12, ha='left')
        plt.figtext(0.1, -0.10, f"User with the most creative contributions: {user_most_creative_contributions} ({creative_contributions_count} actions)", fontsize=12, ha='left')
        plt.figtext(0.1, -0.15, f"User with the lowest contributions in total: {user_least_contributions} ({least_contributions_count} actions)", fontsize=12, ha='left')
        plt.figtext(0.1, -0.20, f"User with the most administrative actions: {user_most_administrative_contributions} ({administrative_contributions_count} actions)", fontsize=12, ha='left')
        plt.figtext(0.1, -0.25, f"User with the most viewing actions: {user_most_viewing_contributions} ({viewing_contributions_count} actions)", fontsize=12, ha='left')

        plt.show()
        fig.savefig(save_path, format='png', bbox_inches='tight')  # Save the plot as an image

# Function to display the plot and send email button
def display_plot_with_send_email(plot_function, save_path, subject, description):
    """
    This function displays a plot and a button to send the plot via email.

    It calls the specified plot function to create the plot, and then displays the send email controls.
    """
    clear_output()
    plot_function(save_path)  # Create the plot and save it as an image
    send_email_controls = create_send_email_button(save_path, subject)
    page = create_page(subject, description, additional_widgets=[plot_output, send_email_controls])
    display(page)

# Functions to display different pages
def display_actions_by_user():
    """
    This function displays the page for plotting actions by user.

    It creates the plot and displays the send email controls.
    """
    display_plot_with_send_email(plot_actions_by_user, 'actions_by_user.png', 'Actions by User', actions_by_user_description)

def display_actions_by_document():
    """
    This function displays the page for plotting actions by document.

    It creates the plot and displays the send email controls.
    """
    display_plot_with_send_email(plot_actions_by_document, 'actions_by_document.png', 'Actions by Document', actions_by_document_description)

def display_number_of_actions_over_time():
    """
    This function displays the page for plotting the number of actions over time.

    It creates the plot and displays the send email controls.
    """
    display_plot_with_send_email(plot_number_of_actions_over_time, 'number_of_actions_over_time.png', 'Number of Actions Over Time', actions_over_time_description)

def display_actions_by_tab():
    """
    This function displays the page for selecting a document to plot actions by tab.

    It shows the title and document selection dropdown.
    """
    clear_output()
    display_document_selection()

def display_progress_patterns():
    """
    This function plots the progress pattern of new tabs added by users over time
    """
    display_plot_with_send_email(plot_progress_pattern, 'progress_patterns.png', 'Progress Patterns', progress_patterns_description)

def display_collaboration_signs():
    """
    This function plots the collaboration signs of the users
    """
    display_plot_with_send_email(plot_collaboration_signs, 'collaboration_signs.png', 'Collaboration Signs', progress_patterns_description)

def display_working_hours():
    """
    This function plots the working hours of the users
    """
    display_plot_with_send_email(plot_working_hours, 'working_hours.png', 'Working Hours', working_hours_description)

def display_user_contributions():
    """
    This function plots the user contributions to the progress of the task
    """
    display_plot_with_send_email(plot_user_contributions, 'user_contributions.png', 'User Contributions', user_contributions_description)

def display_student_activities():
    """
    This function plots the main activities of the users
    """
    display_plot_with_send_email(plot_student_activities, 'student_activities.png', 'Student Activities', student_activities_description)


###################################
##### CHAT BOT FUNCTIONALLITY #####
###################################
# Helper function to get top actions for a specific user
def get_top_actions(user, data, n=5):
    user_actions = Counter([item.get('Description') for item in data.values() if item.get('User') == user])
    top_actions = user_actions.most_common(n)
    return ', '.join([action[0] for action in top_actions])

# Define patterns and responses
def set_patterns(total_actions, users, documents, tabs, descriptions, unique_users, unique_documents):
    patterns = [
        (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Welcome to the project management assistant.']),
        (r'how are you?', ['I\'m functioning well, thank you!', 'I\'m operational and ready to assist with your project management.']),
        (r'what is your name?', ['I\'m the Project Management Assistant.', 'You can call me the Assistant.']),
        (r'how do I use context in my project?', ['You can use context to manage state and share data across different parts of your application.']),
        (r'how do I define a type?', ['Defining types helps ensure that your variables are used correctly and can prevent errors in your application.']),
        (r'how can I use a keyboard shortcut?', ['Keyboard shortcuts can be defined to improve the efficiency of your application, allowing users to perform actions quickly.']),
        (r'how do I plan my project?', ['Planning your project involves setting clear goals, defining tasks, and creating a timeline to ensure that everything is completed on time.']),
        (r'how do I assemble my project components?', ['Assembling your project components involves integrating different parts of your application to work together seamlessly.']),
        (r'how can I sketch my ideas?', ['Sketching your ideas can help visualize your project before implementation, making it easier to plan and execute.']),
        (r'how do I draw diagrams for my project?', ['Drawing diagrams can help illustrate the structure and flow of your application, making it easier to understand and communicate.']),
        (r'what is a part in project management?', ['A part in project management refers to a component or segment of the overall project, which can be managed and tracked individually.']),
        (r'what is a studio in the context of development?', ['A studio in development refers to an integrated environment where various aspects of a project can be designed, developed, and tested.']),
        (r'exit|bye|goodbye', ['Thank you for using the Project Management Assistant. Goodbye!', 'Farewell! Don\'t hesitate to return if you need more assistance with your project.']),
        (r'How many total actions are recorded?', [f"There are {total_actions} total actions recorded in the database."]),
        (r'Who is the most active user?', [f"The most active user is {users.most_common(1)[0][0]} with {users.most_common(1)[0][1]} actions."]),
        (r'How many unique users are there?', [f"There are {unique_users} unique users in the database."]),
        (r'What is the most frequently accessed document?', [f"The most frequently accessed document is '{documents.most_common(1)[0][0]}' with {documents.most_common(1)[0][1]} accesses."]),
        (r'How many different documents were accessed?', [f"There were {unique_documents} different documents accessed."]),
        (r'What is the most common tab used?', [f"The most commonly used tab is '{tabs.most_common(1)[0][0]}' with {tabs.most_common(1)[0][1]} uses."]),
        (r'What is the most frequent action description?', [f"The most frequent action description is '{descriptions.most_common(1)[0][0]}' occurring {descriptions.most_common(1)[0][1]} times."]),
        (r'What are the top 3 most active users?', [f"The top 3 most active users are: 1. {users.most_common(3)[0][0]} ({users.most_common(3)[0][1]} actions), 2. {users.most_common(3)[1][0]} ({users.most_common(3)[1][1]} actions), and 3. {users.most_common(3)[2][0]} ({users.most_common(3)[2][1]} actions)."]),
        (r'What is the least used tab?', [f"The least used tab is '{tabs.most_common()[-1][0]}' with only {tabs.most_common()[-1][1]} uses."]),
        (r'How many different types of actions descriptions are there?', [f"There are {len(descriptions)} different types of actions (unique descriptions) in the database."]),
    ]
    return patterns

# Helper function to handle specific user queries
def handle_specific_user_query(user_input, users, data):
    if "actions did" in user_input and "perform" in user_input:
        user = user_input.split("actions did ")[1].split(" perform")[0]
        return f"{user} performed actions like {get_top_actions(user, data)}"
    elif "documents were accessed by" in user_input:
        user = user_input.split("documents were accessed by ")[1].split("?")[0]
        user_docs = [item.get('Document') for item in data.values() if item.get('User') == user]
        return f"{user} accessed the following documents: {', '.join(user_docs)}"
    elif "total number of actions performed by" in user_input:
        user = user_input.split("total number of actions performed by ")[1].split("?")[0]
        return f"{user} performed a total of {users[user]} actions."
    elif "tab was used the least by" in user_input:
        user = user_input.split("tab was used the least by ")[1].split("?")[0]
        user_tabs = Counter([item.get('Tab') for item in data.values() if item.get('User') == user])
        least_used_tab = user_tabs.most_common()[-1][0]
        return f"The least used tab by {user} is {least_used_tab} with only {user_tabs[least_used_tab]} uses."
    return None

def display_virtual_assistant():
    """
    This function displays the page for the personal virtual assistant.
    """
    clear_output()
    virtual_assistant_description = "This is your project management assistant. Ask me anything about your project's data."
    page = create_page('Personal Virtual Assistant', virtual_assistant_description, additional_widgets=None)
    display(page)

    # Fetch data from Firebase
    _, data = fetch_data_from_firebase()

    # Process the data
    users = Counter([item.get('User') for item in data.values()])
    documents = Counter([item.get('Document') for item in data.values() if item.get('Document')])
    tabs = Counter([item.get('Tab') for item in data.values() if item.get('Tab')])
    descriptions = Counter([item.get('Description') for item in data.values() if item.get('Description')])

    # Calculate some statistics
    total_actions = len(data)
    unique_users = len(users)
    unique_documents = len(documents)
    unique_tabs = len(tabs)

    # Set patterns for the chatbot
    patterns = set_patterns(total_actions, users, documents, tabs, descriptions, unique_users, unique_documents)
    chatbot = Chat(patterns, reflections)

    # Create interactive elements for the chat
    user_input_widget = widgets.Text(placeholder='Type your question here...')
    send_button = widgets.Button(description='Send')
    chat_output = widgets.Output()

    # Display the initial greeting
    with chat_output:
        print("Hello! I'm the Project Management Assistant. How can I help you today?")

    def on_send_button_clicked(b):
        user_input = user_input_widget.value
        user_input_widget.value = ''

        with chat_output:
            if user_input.lower() in ['exit', 'bye', 'goodbye']:
                print("ChatBot: Thank you for using the Project Management Assistant. Goodbye!")
                return

            print(f"You: {user_input}")
            response = chatbot.respond(user_input)
            if response:
                print("ChatBot:", response)
            else:
                specific_response = handle_specific_user_query(user_input, users, data)
                print("ChatBot:", specific_response if specific_response is not None else "This type of question is not supported.")

    send_button.on_click(on_send_button_clicked)

    # Layout for the chat interface
    chat_interface = widgets.VBox([user_input_widget, send_button, chat_output], layout=widgets.Layout(align_items='center'))
    display(chat_interface)

# Initial display of the main dashboard
display_main_dashboard()


Box(children=(VBox(children=(HTML(value="\n        <div style='background: linear-gradient(to right, #ffffff, …